In [1]:
# Load necessary packages
import torch
from tqdm import tqdm
from calculate_topk import generate_branching_responses, print_responses
from huggingface_hub import notebook_login
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
notebook_login()

In [2]:
from javascript import require, On, Once, AsyncTask, once, off, globalThis
import json
import logging
import time
import vision

import building

MINECRAFT_VERSION = '1.20.2'

craft = require('./control_primitives/craftItem.js')
globalThis.craft = craft
explore = require('./control_primitives/exploreUntil.js')
globalThis.explore = explore
move = require('./control_primitives/goToPosition.js')
globalThis.move = move
kill = require('./control_primitives/killMob.js')
globalThis.kill = kill
collectPosition = require('./control_primitives/mineBlockPosition.js')
globalThis.collectPosition = collectPosition
collectType = require('./control_primitives/mineBlockType.js')
globalThis.collectType = collectType
pickupDroppedItem = require('./control_primitives/pickupDroppedItem.js')
globalThis.pickupDroppedItem = pickupDroppedItem
place = require('./control_primitives/placeItem.js')
globalThis.place = place
shoot = require('./control_primitives/shoot.js')
globalThis.shoot = shoot
smelt = require('./control_primitives/smeltItem.js')
globalThis.smelt = smelt
chest = require('./control_primitives/useChest.js')
globalThis.chest = chest

Vec3 = require('vec3').Vec3
globalThis.Vec3 = Vec3
mineflayer = require('mineflayer')
globalThis.mineflayer = mineflayer
pathfinder = require('mineflayer-pathfinder')
globalThis.pathfinder = pathfinder
pvp = require('mineflayer-pvp').plugin
globalThis.pvp = pvp
mcData = require('minecraft-data')(MINECRAFT_VERSION)
globalThis.mcData = mcData
armorManager = require("mineflayer-armor-manager")
globalThis.armorManager = armorManager
autoeat = require('mineflayer-auto-eat').plugin
globalThis.autoeat = autoeat
collectblock = require('mineflayer-collectblock').plugin
globalThis.collectblock = collectblock
hawkeye = require('minecrafthawkeye')
globalThis.hawkeye = hawkeye
toolPlugin = require('mineflayer-tool').plugin
globalThis.toolPlugin = toolPlugin
# mineflayerViewer = require('prismarine-viewer')

RANGE_GOAL = 1

# Fixing some crafting recipes involving planks
all_planks = [name for name in mcData.blocksByName if 'planks' in name]
crafting_table_list = []
bookshelves_list = []

stick_list = [] # [{ 'inShape': [int(mcData.itemsByName['bamboo']['id']), int(mcData.itemsByName['bamboo']['id'])], 'result': { 'id': 807, 'count': 1 } }]

shield_list = []
wood_sword_list = []
wood_pickaxe_list = []
wood_axe_list = []
wood_shovel_list = []
wood_hoe_list = []

for name in all_planks:
    crafting_table_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])]], 'result': { 'id': 278, 'count': 1 } })
    bookshelves_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName['book']['id']), int(mcData.itemsByName['book']['id']), int(mcData.itemsByName['book']['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])]], 'result': { 'id': 264, 'count': 1 } })
    
    stick_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id'])]], 'result': { 'id': 807, 'count': 4 } })
    
    shield_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName['iron_ingot']['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [None, int(mcData.itemsByName[name]['id']), None]], 'result': { 'id': 1116, 'count': 1 } })
    wood_sword_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 777, 'count': 1 } })
    wood_pickaxe_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [None, int(mcData.itemsByName['stick']['id']), None], [None, int(mcData.itemsByName['stick']['id']), None]], 'result': { 'id': 779, 'count': 1 } })
    wood_axe_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName['stick']['id'])], [None, int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 780, 'count': 1 } })
    wood_shovel_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName['stick']['id'])], [int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 778, 'count': 1 } })
    wood_hoe_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [None, int(mcData.itemsByName['stick']['id'])], [None, int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 781, 'count': 1 } })

mcData.recipes[mcData.itemsByName['crafting_table']['id']] = crafting_table_list
mcData.recipes[mcData.itemsByName['bookshelf']['id']] = bookshelves_list
mcData.recipes[mcData.itemsByName['stick']['id']] = stick_list
mcData.recipes[mcData.itemsByName['shield']['id']] = shield_list
mcData.recipes[mcData.itemsByName['wooden_sword']['id']] = wood_sword_list
mcData.recipes[mcData.itemsByName['wooden_pickaxe']['id']] = wood_pickaxe_list
mcData.recipes[mcData.itemsByName['wooden_axe']['id']] = wood_axe_list
mcData.recipes[mcData.itemsByName['wooden_shovel']['id']] = wood_shovel_list
mcData.recipes[mcData.itemsByName['wooden_hoe']['id']] = wood_hoe_list



# Load bot
def load_bot(username=None):
  # Create bot
  if not username:
    random_number = 176
    username = f'poo_bucket{random_number}'

  bot = mineflayer.createBot({ 
      'host': 'localhost',
      'port': 22222,
      'username': username, 
      'hideErrors': False 
  })

  once(bot, 'login')
  bot.chat('I spawned')
  #   mineflayerViewer(bot, { 'port': 3000, 'firstPerson': True })

  # Load plugins
  bot.loadPlugin(collectblock)
  bot.loadPlugin(hawkeye)
  bot.loadPlugin(pathfinder.pathfinder)
  bot.loadPlugin(pvp)
  bot.loadPlugin(toolPlugin)

  globalThis.mcData = mcData
  globalThis.pathfinder = pathfinder
  movements = pathfinder.Movements(bot, mcData)
  bot.pathfinder.setMovements(movements)

  # Auto armor
  bot.loadPlugin(armorManager)
  bot.armorManager.equipAll()

  # Auto eat
  bot.loadPlugin(autoeat)
  bot.autoEat.options.priority = 'foodPoints'
  bot.autoEat.options.startAt = 14
  bot.autoEat.options.bannedFood.push('golden_apple')

  bot.pathfinder.thinkTimeout = 5000

  
  return bot, movements

In [6]:
bot, movements = load_bot('poo_bucket176')
globalThis.bot = bot

In [7]:
import env_info
environment = env_info.getPromptInfo(bot)

In [4]:
# Configure the model to use 4-bit quantization
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto", quantization_config=config)

bin c:\Users\garre\Documents\GitHub\minecraft-agent\venv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [8]:
input_text = f"{environment}... Previous Task: 'Gather Wood'. What should my next task be?"
print(input_text)

{'world_info': {'time': 'It is currently 5998 in the Morning', 'weather': 'It is clear.', 'biome': 'Old Growth Birch Forest'}, 'environment_info': {'mobs': ['sheep is 14 blocks away.'], 'droppedItems': [], 'players': [], 'beds': [], 'chests': []}, 'player_state': {'position': 'Vec3(-78.34306842829018, 71, -4.500335006846927)', 'pitch': 1.3315847535722014e-08, 'yaw': 1.5707960604787876, 'health': 20, 'hunger': 20, 'heldItem': 'Iron Axe', 'inventory': ['Iron Axe x1', 'Oak Log x3']}, 'recent_chat_messages': []}... Previous Task: 'Gather Wood'. What should my next task be?


In [12]:
modules = json.loads(open('modules.json').read())

In [13]:
input_text = f"ENVIRONMENT: {environment} MODULES: {modules}... Current Goal: "

r, p = generate_branching_responses(model, tokenizer, prompt=input_text, num_branches=2, max_length=150)
print_responses(r, p, input_text=input_text)

100%|██████████| 2/2 [00:51<00:00, 25.92s/it]

Response 1 (Avg. Confidence: 0.7481): 
The provided context does not specify which module should be called or how to call it. Therefore, I cannot answer this question from the provided context.

Response 2 (Avg. Confidence: 0.7207): 

The provided context does not specify which module should be called or how to call it. Therefore, I cannot answer this question from the provided context.



In [ ]:
from dataset import Dataset
ds = Dataset.from_dict({"text": observations})
ds

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.15,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj']
)

lora_model = get_peft_model(model, peft_config)
tokenizer.padding_side = 'right'
lora_model.print_trainable_parameters()

In [ ]:
training_arguments = TrainingArguments(
    output_dir="gemma-2b-SH-it-pecey-fan",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=1,
    num_train_epochs=3,
    push_to_hub=False
)

In [ ]:
trainer = SFTTrainer(
    model=lora_model,
    train_dataset=ds,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
)